In [2]:
import torch
import torch.nn as nn

In [3]:
GPT_CONFIG_124M = {
    "vocab_size" : 50257,
    "context_length": 1024, 
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

In [26]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(*[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias = False)


    def forward(self, in_idx):
        batch_size, seq_len, = in_idx.shape
        tok_embs = self.tok_emb(in_idx)
        pos_embs = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embs + pos_embs
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits 


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x


class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift
    

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2) / torch.pi) * (x + 0.044715 * torch.pow(x,3))))
    

        




In [27]:
batch_example = torch.randn(2, 5)


ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
out_ln = GELU(out_ln)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

TypeError: GELU.__init__() takes 1 positional argument but 2 were given

In [44]:
g = GELU()
g(batch_example)

tensor([[-0.0000, -0.1144, -0.1327,  0.5459, -0.1018],
        [ 0.3961, -0.0981,  1.3876, -0.0346, -0.0108]])

In [15]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
text1 = "hello my name is"
text2 = "every day is a"

batch.append(torch.tensor(tokenizer.encode(text1)))
batch.append(torch.tensor(tokenizer.encode(text2)))
batch = torch.stack(batch, dim=0)
print(batch)


tensor([[31373,   616,  1438,   318],
        [16833,  1110,   318,   257]])


In [8]:
torch.manual_seed(123)
GPT = DummyGPTModel(GPT_CONFIG_124M)
logits = GPT(batch)
#print(logits)

print(logits)
print(logits.shape)

tensor([[[-0.1338,  1.1018, -0.1079,  ..., -0.8242, -0.3032, -0.1015],
         [-0.9641,  0.3431, -1.6978,  ...,  0.6259, -0.3982,  1.6753],
         [-0.1668,  1.0426,  0.6369,  ...,  0.4858,  0.1500, -0.4176],
         [ 0.0771, -0.0927, -0.5684,  ...,  1.3646, -0.3133, -0.2629]],

        [[-1.0406,  0.1050, -0.6027,  ..., -1.4415,  0.1323,  0.5012],
         [-0.7860,  0.5581, -0.0610,  ...,  0.4835, -0.0077,  1.6621],
         [ 0.7233,  0.1566, -0.3828,  ...,  0.5961, -0.2573, -0.6953],
         [-0.2407, -0.7349, -0.5102,  ...,  2.0057, -0.3694,  0.1814]]],
       grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 4, 50257])
